In [1]:
# Bibliotecas padrão
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import urllib.parse

# Módulos para a parte espacial
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
from shapely import wkt

In [2]:
# Leitura do CSV
df_ptds = pd.read_csv(r"./data/perc_cp_PTD.csv")

# Converter WKT para geometria
df_ptds["geometry"] = df_ptds["geometry"].apply(wkt.loads)

# Criar GeoDataFrame e definir o CRS original correto
df_ptds = gpd.GeoDataFrame(df_ptds, geometry='geometry', crs="EPSG:3857")

# Converter para EPSG:4326 se necessário
df_ptds = df_ptds.to_crs("EPSG:4326")

# Visualização
print(df_ptds.head())

   index_cp cod_postal          dd_desig  \
0     16910   2520-627            Leiria   
1     97890   4910-477  Viana do Castelo   
2     97863   4910-443  Viana do Castelo   
3     97842   4910-384  Viana do Castelo   
4     18901   2560-574            Lisboa   

                                            cc_desig   llll  \
0  Peniche                                       ...  35331   
1  Caminha                                       ...  11613   
2  Caminha                                       ...  11613   
3  Caminha                                       ...  11613   
4  Torres Vedras                                 ...  47802   

             LOCALIDADE   CP4  CP3                 CPALF   LATITUDE  \
0               Peniche  2520  627               PENICHE  39.361790   
1  Vila Praia de Âncora  4910  477  VILA PRAIA DE ÂNCORA  41.824709   
2  Vila Praia de Âncora  4910  443  VILA PRAIA DE ÂNCORA  41.825478   
3  Vila Praia de Âncora  4910  384  VILA PRAIA DE ÂNCORA  41.825204   
4

In [5]:
df_consumos = pd.read_csv(r".\data\consumos_por_codigos_postais_allpt.csv")
df_consumos["datahora"] = pd.to_datetime(df_consumos["datahora"])
df_consumos.head()

,consumo,dt_consumo,datahora,codigo_4,codigo_3,hr_consumo,codigo_postal,faixa_codigo_postal
0,119.721454,2024-02-29,2024-02-29 20:00:00+00:00,1750.0,399.0,20:00,1750-399,1000 a 2000
1,43.627310,2024-02-14,2024-02-14 23:00:00+00:00,1700.0,146.0,23:00,1700-146,1000 a 2000
2,251.238994,2024-02-19,2024-02-19 08:00:00+00:00,1070.0,26.0,08:00,1070-026,1000 a 2000
3,54.611730,2024-02-21,2024-02-21 22:00:00+00:00,1600.0,57.0,22:00,1600-057,1000 a 2000
4,64.653863,2024-02-24,2024-02-24 10:00:00+00:00,1050.0,48.0,10:00,1050-048,1000 a 2000


In [7]:
print(df_ptds.columns)

Index(['index_cp', 'cod_postal', 'dd_desig', 'cc_desig', 'llll', 'LOCALIDADE',
       'CP4', 'CP3', 'CPALF', 'LATITUDE', 'LONGITUDE', 'scannedAddress',
       'index_ptd', 'area_ptd_m2', 'geometry', 'area_intersect_m2',
       'area_cp_m2_cp', 'percent_area_in_ptd'],
      dtype='object')


In [8]:
print(df_consumos.columns)

Index(['consumo', 'dt_consumo', 'datahora', 'codigo_4', 'codigo_3',
       'hr_consumo', 'codigo_postal', 'faixa_codigo_postal'],
      dtype='object')


In [14]:
# 1. Obter intervalo completo de horas no mês (min-max de datahora)
hora_inicio = df_consumos['datahora'].min().floor('H')
hora_fim    = df_consumos['datahora'].max().ceil('H')
todas_horas = pd.date_range(start=hora_inicio, end=hora_fim, freq='H')

# 2. Obter todos os index_ptd únicos
todos_ptds = df_ptds['index_ptd'].unique()

# 3. Criar MultiIndex com todas as combinações possíveis (produto cartesiano)
idx_completo = pd.MultiIndex.from_product(
    [todos_ptds, todas_horas],
    names=['index_ptd', 'datahora']
)

# 4. Definir o índice do DataFrame original e reindexar com o MultiIndex completo
df_completo = (
    df_consumo_horario_ptd
      .set_index(['index_ptd', 'datahora'])
      .reindex(idx_completo)
      .reset_index()
)

C:\Users\toto9\AppData\Local\Temp\ipykernel_17264\1066736332.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hora_inicio = df_consumos['datahora'].min().floor('H')
C:\Users\toto9\AppData\Local\Temp\ipykernel_17264\1066736332.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hora_fim    = df_consumos['datahora'].max().ceil('H')
C:\Users\toto9\AppData\Local\Temp\ipykernel_17264\1066736332.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  todas_horas = pd.date_range(start=hora_inicio, end=hora_fim, freq='H')


In [16]:
# Verifica quais index_ptd têm pelo menos um valor nulo no consumo_horario
ptd_com_nulos = df_completo.groupby('index_ptd')['consumo_horario'].apply(lambda x: x.isnull().any())

# Calcula a proporção (normalize=True) de index_ptd com valores nulos
proporcao_nulos = ptd_com_nulos.value_counts(normalize=True).get(True, 0.0)

print(f'Proporção de index_ptd com valores nulos: {proporcao_nulos:.2%}')

Proporção de index_ptd com valores nulos: 100.00%


In [17]:
df_completo.head()

,index_ptd,datahora,consumo_horario
0,0,2024-02-01 00:00:00+00:00,NaN
1,0,2024-02-01 01:00:00+00:00,NaN
2,0,2024-02-01 02:00:00+00:00,NaN
3,0,2024-02-01 03:00:00+00:00,NaN
4,0,2024-02-01 04:00:00+00:00,NaN


In [21]:
# Verifica quais index_ptd têm pelo menos 5 valores nulos no consumo_horario
ptd_com_5_nulos = df_completo.groupby('index_ptd')['consumo_horario'].apply(lambda x: x.isnull().sum() >= 35)

# Calcula a proporção (normalize=True) de index_ptd com 5 ou mais valores nulos
proporcao_5_nulos = ptd_com_5_nulos.value_counts(normalize=True).get(True, 0.0)

print(f'Proporção de index_ptd com >=35 valores nulos: {proporcao_5_nulos:.8%}')

Proporção de index_ptd com >=35 valores nulos: 100.00000000%
